In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter, butter

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [4]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [5]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
def plot_T_and_PM_Init_Inje_Evol(file_dir2,file_name,fig_name,**kwargs):
    
    # ~ xlim1 = (-0.1,6)
    # ~ ylim1 = (0.5*1e-3,5e3)
    # ~ ylim2 = (-2,120)
            
    xlim1 = kwargs.get('xlim1', (-0.1,6))
    ylim1 = kwargs.get('ylim1', (0.5*1e-3,2e4))
    ylim2 = kwargs.get('ylim2', (-2,50))
    
    i_aux = file_name.find('_N')
    # file0 = 'Temp_3D_Harmo'+ file_name[i_aux:]
    file1 = 'SimuType0'    + file_name[i_aux:] ########## file1 = 'SimuType0'    + file_name[i_aux:17+36]
    file2 = 'SimuType4_01'  + file_name[i_aux:]
    file3 = 'SimuType2_01'  + file_name[i_aux:]

    # tt0, T_CM0, T_aux0, PM0 = load_T_and_PM_simu(file_dir2+file0)
    tt1, T_CM1, T_aux1, PM1 = load_T_and_PM_simu(file_dir2+'Temp_'+file1)
    tt2, T_CM2, T_aux2, PM2 = load_T_and_PM_simu(file_dir2+'Temp_'+file2+'50eV')
    tt3, T_CM3, T_aux3, PM3 = load_T_and_PM_simu(file_dir2+'Temp_'+file3+'50eV')

    aux = mean(PM1[-100:])
    PM_variation = ( aux - mean(PM3[-100:]) ) / aux
    T_variation  = mean(T_aux3[-100:,0]) + mean(T_aux3[-100:,1]) + mean(T_aux3[-100:,2])
            

    # Auxiliary arrays:
    t_aux1 = array([tt2[ 0],tt2[ 0]])
    t_aux2 = array([tt2[-1],tt2[-1]])
    y1_aux = array([1.0e-3 ,1.0e-1 ])
#     y2_aux = array([0 ,20 ])
    y2_aux = array([0 ,50 ])

    tt    = concatenate( (   tt1,   tt2,   tt3) )
    T_CM  = concatenate( (T_CM1,T_CM2,T_CM3) )
    T_aux = concatenate( (T_aux1,T_aux2,T_aux3) )
    PM    = concatenate( (PM1,PM2,PM3) )
    SNR = np.abs( aux - mean(PM3[-100:]) )/np.sqrt(aux) # Sig-to-Noi ratio considering Poisson noise
        
    temps = [tt,t_aux1, t_aux2]
    temperature = [T_aux, T_variation, T_CM]
    fluo = [PM, PM_variation, SNR]
        
    return temps, temperature, fluo

In [8]:
def load_Temp_init_bin_Lan(str_load,flag_print):   
    
    fid = open(str_load+'.bin', 'rb')
    
    a = fromfile(fid, int32,1)        # Read record start tag
    aux  = fromfile(fid, int32,1)
    junk = fromfile(fid, int32,1)        # Read record stop tag
    N_ions = aux[0]

    b = fromfile(fid, int32,1)        # Read record start tag
    aux  = fromfile(fid, int32,1)
    junk = fromfile(fid, int32,1)        # Read record stop tag
    j_save = aux[0]
    
    c = fromfile(fid, int32,1)        # Read record start tag
    aux  = fromfile(fid, float64,1)
    junk = fromfile(fid, int32,1)        # Read record stop tag
    dt_j_save_next = aux[0]

    d = fromfile(fid, int32,1)        # Read record start tag
    aux  = fromfile(fid, float64,1)
    junk = fromfile(fid, int32,1)        # Read record stop tag
    eta = aux[0]

    e = fromfile(fid, int32  ,1   )        # Read record start tag
    Temp = fromfile(fid, float64,1)
    junk = fromfile(fid, int32  ,1   )        # Read record stop tag
    
    f = fromfile(fid, int32  ,1   )        # Read record start tag
    save_T = fromfile(fid, float64,7*j_save)
    junk = fromfile(fid, int32  ,1   )        # Read record stop tag
    
    fid.close
    
    if flag_print == 1:
        print(a,b,c,d,e,f)
        print('N_ions',N_ions)
        print('j_save',j_save)
        print('dt_j_save_next',dt_j_save_next)
        print('eta',eta)
        print('Temp',Temp)
    
    
    save_T = reshape(save_T,(j_save,7),order='F')
    print(save_T[0])

    return N_ions, j_save, dt_j_save_next, eta, Temp, save_T

# Sélection fichier

In [9]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : D:/CIMLuser/adrien/Simulations/20200710/DC03_RF09/Try00
> Filename : Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 11, 18, 30, 39, 49]
> slashcond | -2
Points deleted because they were not complete ['DC09_RF11']   5 pt(s)
Total number of data directories 72


In [10]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 12
000 > DC03_RF09
001 > DC03_RF10
002 > DC03_RF11
003 > DC05_RF07
004 > DC05_RF09
005 > DC05_RF10
006 > DC05_RF11
007 > DC07_RF09
008 > DC07_RF10
009 > DC07_RF11
010 > DC09_RF09
011 > DC09_RF10
> N_ions = 1024
> e_GMol = 50


In [11]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg) # stopping_power

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 12
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05']
Hello
Point n° 0
00 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try00/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 01  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try01/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 02  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try02/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 03  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try03/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
00 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.2000D+01V_D1.0_S1.0RFG
Point n° 1
01 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC03_RF10\Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.2000D+01V_

10 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF09\Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.5000D+01V_D1.0_S1.0RFG
10 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF09\Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.5000D+01V_D1.0_S1.0RFG
Point n° 11
11 - 00  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 01  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try01/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 02  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try02/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 03  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try03/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 04  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try04/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
11 - 05  >  D:/CIMLuser/adrien/Simulations/20200710\DC09_RF10\Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.500

In [12]:
# Where SNR for each try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(list(points_and_coord.keys()))
print(mean(trig_detection,axis=1))

print(len(trig_detection))

['DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF07', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF09', 'DC09_RF10']
[0.83333333 0.         0.         0.         0.33333333 0.
 0.         0.         0.         0.         0.         0.        ]
12


In [14]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [15]:
## Data loading with hardcode names

plot_point = 0
plot_try = 1
dirname_hc = str(data0[plot_point][plot_try])+'/'

datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5'
data0_hc    = '/Temp_SimuType0_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV'

print(dirname_hc)
print(datainit_hc)

D:/CIMLuser/adrien/Simulations/20200710\DC03_RF09\Try01/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [16]:
N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print = 1)

print('N_ions',N_ions)
print('j_save',j_save)
print('dt*j_save_next',dt_j_save_next)
print('eta',eta)
print('Temp',Temp)
print('save_T',save_T)
print('shape(save_T)',shape(save_T))

#  Sacahnt que dans le Fortran
# dt   = 2d-9
# j_save_next  = 100
# eta = 1.5d-21
# Temperature = 0.5d-3

[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 6.92704091e-04 1.01630676e-02 3.22024709e-06
 1.69195590e+01 3.90546636e+00 7.46999518e+01]
N_ions 1024
j_save 1980
dt*j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
save_T [[5.05000000e-07 6.92704091e-04 1.01630676e-02 ... 1.69195590e+01
  3.90546636e+00 7.46999518e+01]
 [1.01000000e-06 1.04465106e-03 1.02405527e-02 ... 6.13178702e-01
  1.54857708e+00 1.10191533e+02]
 [1.51500000e-06 1.07668206e-04 2.83482578e-03 ... 2.90957551e+01
  3.19207632e+01 6.61002620e+01]
 ...
 [9.98890000e-04 4.37278441e-11 9.01850226e-08 ... 2.89355474e-04
  2.88426889e-04 5.43003396e-04]
 [9.99395000e-04 4.98293214e-09 3.15528778e-08 ... 2.84604150e-04
  2.92778910e-04 4.91628909e-04]
 [9.99900000e-04 8.95899571e-10 2.42890493e-08 ... 2.91364086e-04
  2.71358631e-04 4.96362702e-04]]
shape(save_T) (1980, 7)


In [17]:
dirname_hc[40:49]

'DC03_RF09'

In [18]:
# Get T and fluo PM
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],'coucou' )

# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [ ]:
n_dt = 100

i_free__fly_Lan = 2*1e3 
i_free__fly = 100
i_therm_fly = 2000
i_laser_fly = 1000*3
i_cool__fly = 2000


dt = 1/2e6
print(dt)

In [21]:
# Plot temperature Langevin + md laser

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]


#fig_name = file_name[-9:]
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,75,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,75,color='orange')
time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
    # size=24, ha='left', va='top', color='xkcd:azul',
    # bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

In [202]:
# Plot all temp rf relax après refroidissement

fig, ax = plt.subplots(3, 2,num='Temp interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0

for l in range(0,2):
    for k in range(0,3):
        
        plot_point = 0
        plot_try = to_plot
        dirname_hc = str(data0[plot_point][plot_try])+'/'
        N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
        load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
        # Get T and fluo PM
        onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
        ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],'coucou' )
        
        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run
        tt_md = ta[0]
        T_aux = te[0]

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        PM = ti[0]
        
        ax[k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
        ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
        ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
        time_switch = i_free__fly_Lan*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
        ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 6:
            break
    if to_plot == 6:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.suptitle(dirname_hc[40:49])
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 1.01548937e-03 6.71730325e-02 2.29217981e-06
 1.79474787e+01 4.37648391e+00 7.91182553e+01]
[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 6.92704091e-04 1.01630676e-02 3.22024709e-06
 1.69195590e+01 3.90546636e+00 7.46999518e+01]
[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 5.72896435e-05 2.57597598e-03 7.79515994e-06
 1.74460290e+01 4.46020910e+00 7.61256628e+01]
[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 9.55335681e-04 1.08380207e-02 3.65433681e-06
 1.73907886e+01 3.81140621e+00 7.80533896e+01]
[4] [4] [8] [8] [8] [110880]
N_ions 1024
j_save 1980
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 2.69208421e-03 2.23136037e-02 1.50363249e-07
 1.7873

In [27]:
# Plot all fluo rf relax après refroidissement

fig, ax = plt.subplots(3, 2,num='Fluo interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

plot_point = 4
to_plot = 0
for l in range(0,2):
    for k in range(0,3):       
        
        plot_try = to_plot
        dirname_hc = str(data0[plot_point][plot_try])+'/'
        N_ions, j_save, dt_j_save_next, eta, Temp, save_T =\
        load_Temp_init_bin_Lan(dirname_hc+datainit_hc,0)
        # Get T and fluo PM
        onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
        ta, te, ti = plot_T_and_PM_Init_Inje_Evol(dirname_hc,sort(onlyfiles)[0].strip('.dat')[4:],'coucou' )
        
        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run
        tt_md = ta[0]
        T_aux = te[0]

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        PM = ti[0]
        
        ax[k,l].plot(tt_md*1e3,PM, label='Tx',color='C0')
        ax[k,l].vlines(ta[1]*1e3,1e-4,75,color='red')
        ax[k,l].vlines(ta[2]*1e3,1e-4,75,color='red')
        time_switch = i_free__fly_Lan*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,75,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,75,color='orange')
        time_switch = (i_free__fly_Lan+2*i_laser_fly)*dt
        ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,75,color='purple')
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 6:
            break
    if to_plot == 6:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.suptitle(dirname_hc[40:49])
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

[5.05000000e-07 2.28687012e-03 1.35036885e-02 3.06969448e-07
 1.78312287e+01 3.51932927e+00 7.40049048e+01]
[5.05000000e-07 1.94285146e-03 2.42954029e-02 4.62818009e-05
 1.76018874e+01 3.61204193e+00 7.63659575e+01]
[5.05000000e-07 1.84424710e-04 1.50256642e-02 9.02062378e-05
 1.78325688e+01 4.40049629e+00 7.36693518e+01]
[5.05000000e-07 6.71361506e-04 9.38661529e-02 2.47417586e-06
 1.78838150e+01 3.69519777e+00 7.38199215e+01]
[5.05000000e-07 1.06570719e-03 8.68533737e-04 1.08323857e-04
 1.73173273e+01 3.97396542e+00 7.31246142e+01]
[5.05000000e-07 1.55465301e-04 1.61923536e-02 2.66925370e-06
 1.82890136e+01 4.28486244e+00 7.47808802e+01]


In [ ]:
# plot ion cloud XYZ

print(plot_subdir)
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')